In [43]:
import sys
import win32com.client
import subprocess
import time
import os

# Check a process is running
def process_exists(process_name):   
    call = 'TASKLIST', '/FI', 'imagename eq %s' % process_name
    # use buildin check_output right away
    output = subprocess.check_output(call).decode()
    # check in last line for process name
    last_line = output.strip().split('\r\n')[-1]
    # because Fail message could be translated
    return last_line.lower().startswith(process_name.lower())

# If SAP is not opened, then open SAP
if not process_exists("saplogon.exe"):
    subprocess.call(r'"C:\Program Files (x86)\SAP\FrontEnd\SAPgui\sapshcut.exe" -system=UP2 -client=100')
    time.sleep(5)

# region
try:

    SapGuiAuto = win32com.client.GetObject("SAPGUI")
    if not type(SapGuiAuto) == win32com.client.CDispatch:
        exit()

    application = SapGuiAuto.GetScriptingEngine
    if not type(application) == win32com.client.CDispatch:
        SapGuiAuto = None
        exit()

    connection = application.Children(0)
    if not type(connection) == win32com.client.CDispatch:
        application = None
        SapGuiAuto = None
        exit()

    session = connection.Children(0)
    if not type(session) == win32com.client.CDispatch:
        connection = None
        application = None
        SapGuiAuto = None
        exit()
        

    try:
        if session.findById("wnd[1]"):
            session.findById("wnd[1]").sendVKey(0)
    except:
        pass

    # vba code
    session.findById("wnd[0]").maximize()
    session.findById("wnd[0]/tbar[0]/okcd").text = "EL31"
    session.findById("wnd[0]").sendVKey(0)
    session.findById("wnd[0]/usr/btn%PS06004_1000").press()
    session.findById("wnd[0]/usr/ctxtSEL_DEV-LOW").text = "7045000"
    session.findById("wnd[0]/usr/ctxtSEL_DEV-LOW").caretPosition = 7
    session.findById("wnd[0]/usr/btn%PS33072_1000").press()
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    session.findById("wnd[0]/usr/cntlBCALVC_EVENT2_D100_C1/shellcont/shell").pressToolbarContextButton("&MB_EXPORT")
    session.findById("wnd[0]/usr/cntlBCALVC_EVENT2_D100_C1/shellcont/shell").selectContextMenuItem("&XXL")
    session.findById("wnd[1]/tbar[0]/btn[0]").press()
    session.findById("wnd[1]/usr/ctxtDY_PATH").text = "c:\\temp"
    session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "EXPORT.XLSX"
    session.findById("wnd[1]/usr/ctxtDY_FILENAME").caretPosition = 11
    session.findById("wnd[1]/tbar[0]/btn[0]").press()

except Exception as e:
    print(sys.exc_info()[0])

finally:
    session = None
    connection = None
    application = None
    SapGuiAuto = None

